# dev explicit dormand prince time stepping

In [1]:
%matplotlib inline
from lib.my_initialization import *
# from lib.controller.one_step_forward_euler import *
%autocall 1
%load_ext autoreload
%autoreload 2

Automatic calling is: Smart


# initialize a tetrahedral mesh

In [27]:
os.chdir(nb_dir)
# input_file_name = f'../data/spherical_meshes/spherical_mesh_64.stl'input_file_name = f'../data/spherical_meshes/spherical_mesh_64.stl'
# input_file_name = f'../data/spherical_meshes/spherical_mesh_1000.stl'
input_file_name = f'../data/spherical_meshes/spherical_mesh_100.stl'
input_file_name = os.path.join(nb_dir,input_file_name)
tme = 0.
mass_density=1.
dict_values_system = initialize_system(input_file_name, time_initial=tme, mass_density=mass_density)
locals().update(dict_values_system)
print(list(dict_values_system.keys()))
tauK = element_array_time
tau = node_array_time

#define Lamé parameters
mu = 1.; lam = 10.; 
#define coefficient for Rayleigh damping
gamma = 0.#1.#1.;

#define adaptive time stepping absolute tolerances
atol_x = 1e-7; atol_v = 1e-7; 
# btol_x = 1e-12; btol_v = 1e-12; 

#get method of computing elastic forces 
zero_mat = np.zeros((4,3))
# calc_P = get_calc_P(mu, lam)
# compute_nodal_damping_forces  = get_compute_nodal_damping_forces(mu,lam,gamma)
# comp_nodal_elastic_forces = get_comp_nodal_elastic_forces(mu, lam)

elements = element_array_index
vertices = node_array_position

N_elements = elements.shape[0]
N_vertices = vertices.shape[0]

#initialize stepsizes of simulation
stepsize_init = 0.001#0.0001 
element_array_stepsize = np.zeros_like(element_array_time) + stepsize_init
element_array_count_calls_one_step = np.zeros_like(element_array_time,dtype=int)
element_array_count_config_updates = np.zeros_like(element_array_time,dtype=int)
momentum = node_array_momentum.copy()
velocities = momentum.copy()
for j in range(N_vertices):
    velocities[j] /= node_array_mass[j]


# #perturb momentum in the x direction and let it run overnight with a small timestep  
v_scale = -1.
velocities[:,0] = v_scale * vertices[:,0].copy()

#initialize containers of measures
volume_lst = []
energy_lst = []
tme_lst    = []
stepsize_mean_lst   = []
stepsize_std_lst    = []
stepsize_median_lst = []

face_normals all zero, ignoring!


['element_array_time', 'element_array_index', 'element_array_mass', 'element_array_volume', 'element_array_inverse_equilibrium_position', 'node_array_equilibrium_position', 'node_array_time', 'node_array_position', 'node_array_momentum', 'node_array_mass', 'node_array_volume']


# get an explicit time step

In [15]:
#do one explicit step
zero_mat = np.zeros((4,3))
# def get_step_to_time(mu,lam,gamma,num_iter = num_iter):
#     local_one_step_implicit_midpoint_rule = get_local_one_step_implicit_midpoint_rule(mu,lam,gamma,num_iter = 30)
#     return local_one_step_implicit_midpoint_rule
# step_to_time = get_step_to_time(mu,lam,gamma,num_iter = num_iter)
compute_one_step_forward_euler_method=get_compute_one_step_forward_euler_method(mu,lam,gamma)
step_to_time = compute_one_step_forward_euler_method

In [21]:
t = 0.01
K_index = 7
Bm = element_array_inverse_equilibrium_position[K_index]
Ka = element_array_index[K_index]
x = vertices[Ka].copy()
v = velocities[Ka].copy()
tau_of_K = element_array_time[K_index]
K_tau = node_array_time[Ka].copy()
K_masses = element_array_mass[Ka]
# step_to_time(t, K_vertices, K_velocities, K_masses, K_tau, tau_of_K, Bm, zero_mat)

x_out, v_out = step_to_time(t,x,v,K_masses,K_tau,tau_of_K,Bm,zero_mat)

def f(t,x,v,K_tau,tau_of_K,K_masses,Bm,zero_mat):
    x_out, v_out = step_to_time(t,x,v,K_masses,K_tau,tau_of_K,Bm,zero_mat)
    return x_out, v_out

# open an existing open source solution

In [2]:
from scipy.integrate import RK45

In [6]:
import inspect

In [9]:
print(inspect.getsource(RK45))

class RK45(RungeKutta):
    """Explicit Runge-Kutta method of order 5(4).

    This uses the Dormand-Prince pair of formulas [1]_. The error is controlled
    assuming accuracy of the fourth-order method accuracy, but steps are taken
    using the fifth-order accurate formula (local extrapolation is done).
    A quartic interpolation polynomial is used for the dense output [2]_.

    Can be applied in the complex domain.

    Parameters
    ----------
    fun : callable
        Right-hand side of the system. The calling signature is ``fun(t, y)``.
        Here ``t`` is a scalar, and there are two options for the ndarray ``y``:
        It can either have shape (n,); then ``fun`` must return array_like with
        shape (n,). Alternatively it can have shape (n, k); then ``fun``
        must return an array_like with shape (n, k), i.e., each column
        corresponds to a single column in ``y``. The choice between the two
        options is determined by `vectorized` argument (see below

In [10]:
class RK45(RungeKutta):
    """Explicit Runge-Kutta method of order 5(4).

    This uses the Dormand-Prince pair of formulas [1]_. The error is controlled
    assuming accuracy of the fourth-order method accuracy, but steps are taken
    using the fifth-order accurate formula (local extrapolation is done).
    A quartic interpolation polynomial is used for the dense output [2]_.

    Can be applied in the complex domain.

    Parameters
    ----------
    fun : callable
        Right-hand side of the system. The calling signature is ``fun(t, y)``.
        Here ``t`` is a scalar, and there are two options for the ndarray ``y``:
        It can either have shape (n,); then ``fun`` must return array_like with
        shape (n,). Alternatively it can have shape (n, k); then ``fun``
        must return an array_like with shape (n, k), i.e., each column
        corresponds to a single column in ``y``. The choice between the two
        options is determined by `vectorized` argument (see below).
    t0 : float
        Initial time.
    y0 : array_like, shape (n,)
        Initial state.
    t_bound : float
        Boundary time - the integration won't continue beyond it. It also
        determines the direction of the integration.
    first_step : float or None, optional
        Initial step size. Default is ``None`` which means that the algorithm
        should choose.
    max_step : float, optional
        Maximum allowed step size. Default is np.inf, i.e., the step size is not
        bounded and determined solely by the solver.
    rtol, atol : float and array_like, optional
        Relative and absolute tolerances. The solver keeps the local error
        estimates less than ``atol + rtol * abs(y)``. Here `rtol` controls a
        relative accuracy (number of correct digits). But if a component of `y`
        is approximately below `atol`, the error only needs to fall within
        the same `atol` threshold, and the number of correct digits is not
        guaranteed. If components of y have different scales, it might be
        beneficial to set different `atol` values for different components by
        passing array_like with shape (n,) for `atol`. Default values are
        1e-3 for `rtol` and 1e-6 for `atol`.
    vectorized : bool, optional
        Whether `fun` is implemented in a vectorized fashion. Default is False.

    Attributes
    ----------
    n : int
        Number of equations.
    status : string
        Current status of the solver: 'running', 'finished' or 'failed'.
    t_bound : float
        Boundary time.
    direction : float
        Integration direction: +1 or -1.
    t : float
        Current time.
    y : ndarray
        Current state.
    t_old : float
        Previous time. None if no steps were made yet.
    step_size : float
        Size of the last successful step. None if no steps were made yet.
    nfev : int
        Number evaluations of the system's right-hand side.
    njev : int
        Number of evaluations of the Jacobian. Is always 0 for this solver as it does not use the Jacobian.
    nlu : int
        Number of LU decompositions. Is always 0 for this solver.

    References
    ----------
    .. [1] J. R. Dormand, P. J. Prince, "A family of embedded Runge-Kutta
           formulae", Journal of Computational and Applied Mathematics, Vol. 6,
           No. 1, pp. 19-26, 1980.
    .. [2] L. W. Shampine, "Some Practical Runge-Kutta Formulas", Mathematics
           of Computation,, Vol. 46, No. 173, pp. 135-150, 1986.
    """
    order = 5
    error_estimator_order = 4
    n_stages = 6
    C = np.array([0, 1/5, 3/10, 4/5, 8/9, 1])
    A = np.array([
        [0, 0, 0, 0, 0],
        [1/5, 0, 0, 0, 0],
        [3/40, 9/40, 0, 0, 0],
        [44/45, -56/15, 32/9, 0, 0],
        [19372/6561, -25360/2187, 64448/6561, -212/729, 0],
        [9017/3168, -355/33, 46732/5247, 49/176, -5103/18656]
    ])
    B = np.array([35/384, 0, 500/1113, 125/192, -2187/6784, 11/84])
    E = np.array([-71/57600, 0, 71/16695, -71/1920, 17253/339200, -22/525,
                  1/40])
    # Corresponds to the optimum value of c_6 from [2]_.
    P = np.array([
        [1, -8048581381/2820520608, 8663915743/2820520608,
         -12715105075/11282082432],
        [0, 0, 0, 0],
        [0, 131558114200/32700410799, -68118460800/10900136933,
         87487479700/32700410799],
        [0, -1754552775/470086768, 14199869525/1410260304,
         -10690763975/1880347072],
        [0, 127303824393/49829197408, -318862633887/49829197408,
         701980252875 / 199316789632],
        [0, -282668133/205662961, 2019193451/616988883, -1453857185/822651844],
        [0, 40617522/29380423, -110615467/29380423, 69997945/29380423]])

NameError: name 'RungeKutta' is not defined

In [11]:
RK45

scipy.integrate._ivp.rk.RK45

In [12]:
from scipy.integrate._ivp import rk

In [14]:
print(inspect.getsource(rk))

import numpy as np
from .base import OdeSolver, DenseOutput
from .common import (validate_max_step, validate_tol, select_initial_step,
                     norm, warn_extraneous, validate_first_step)
from . import dop853_coefficients

# Multiply steps computed from asymptotic behaviour of errors by this.
SAFETY = 0.9

MIN_FACTOR = 0.2  # Minimum allowed decrease in a step size.
MAX_FACTOR = 10  # Maximum allowed increase in a step size.


def rk_step(fun, t, y, f, h, A, B, C, K):
    """Perform a single Runge-Kutta step.

    This function computes a prediction of an explicit Runge-Kutta method and
    also estimates the error of a less accurate method.

    Notation for Butcher tableau is as in [1]_.

    Parameters
    ----------
    fun : callable
        Right-hand side of the system.
    t : float
        Current time.
    y : ndarray, shape (n,)
        Current state.
    f : ndarray, shape (n,)
        Current value of the derivative, i.e., ``fun(x, y)``.
    h : float
        S

# ^that looks complicated... Let's just plug in the simple dormand prince method.

How to read a butcher tableau:
    - https://en.wikipedia.org/wiki/Runge%E2%80%93Kutta_methods#Explicit_Runge.E2.80.93Kutta_methods
The butcher tableau for dormand-prince
    - https://en.wikipedia.org/wiki/Dormand%E2%80%93Prince_method
   

The explicit runge-kutta method is as follows:

$$
y_{n+1} = y_n + h \sum_{i=1}^sb_ik_i
$$

For 
$$k_1 = f(t_n, y_n)$$
$$k_2 = f(t_n+c_2 h, y_n+h(a_{21}k_1))$$
$$k_3 = f(t_n+c_3 h, y_n+h(a_{31}k_1 + a_{32} k_2))$$
$$\vdots$$
$$k_s = f(t_n+c_s h, y_n + h(a_{s1}k_1 + a_{s2}k_2 + \cdots + a_{s,s-1}k_{s-1})$$


- TODO: insert images in avi/doc here for presentation.  they are taken from 
    - https://en.wikipedia.org/wiki/Runge%E2%80%93Kutta_methods#Explicit_Runge.E2.80.93Kutta_methods
    - https://en.wikipedia.org/wiki/Dormand%E2%80%93Prince_method
but could have easily been $\LaTeX$'d<br>

I'm using what's recommended by scipy.  The modified method is presented in [2]

`
References
    ----------
    .. [1] J. R. Dormand, P. J. Prince, "A family of embedded Runge-Kutta
           formulae", Journal of Computational and Applied Mathematics, Vol. 6,
           No. 1, pp. 19-26, 1980.
    .. [2] L. W. Shampine, "Some Practical Runge-Kutta Formulas", Mathematics
           of Computation,, Vol. 46, No. 173, pp. 135-150, 1986.
`

In [72]:
# the stepsize
h = 0.001
#set the current time to the max of the configuration
t = max(np.max(K_tau), tau_of_K)
#explicit newmark method to the current time
k1x, k1v = f(t,x,v,    K_tau,tau_of_K,    K_masses,Bm,zero_mat)
k1x = k1x - x; k1v = k1v - v

In [73]:
#from print(inspect.getsource(scipy.integrate.RK45))
C = np.array([0, 1/5, 3/10, 4/5, 8/9, 1])
A = np.array([
    [0, 0, 0, 0, 0],
    [1/5, 0, 0, 0, 0],
    [3/40, 9/40, 0, 0, 0],
    [44/45, -56/15, 32/9, 0, 0],
    [19372/6561, -25360/2187, 64448/6561, -212/729, 0],
    [9017/3168, -355/33, 46732/5247, 49/176, -5103/18656]
])
B = np.array([35/384, 0, 500/1113, 125/192, -2187/6784, 11/84])
E = np.array([-71/57600, 0, 71/16695, -71/1920, 17253/339200, -22/525,
              1/40])

In [74]:
c2 = C[1]; a21 = A[1,0]
k2x, k2v = f(t+c2*h, x+h*a21*k1x, v+h*a21*k1v,    K_tau,tau_of_K,    K_masses,Bm,zero_mat)
k2x = k2x - x; k2v = k2v - v

In [75]:
c3 = C[2]; a31 = A[2,0]; a32 = A[2,1]
k3x, k3v = f(t+c3*h, x+h*(a31*k1x + a32*k2x), v+h*(a31*k1v + a32*k2v),    K_tau,tau_of_K,    K_masses,Bm,zero_mat)
k3x = k3x - x; k3v = k3v - v

In [76]:
c4 = C[3]; a41 = A[3,0]; a42 = A[3,1]; a43 = A[3,2]
k4x, k4v = f(t+c4*h, x+h*(a41*k1x + a42*k2x + a43*k3x), v+h*(a41*k1v + a42*k2v + a43*k3v),    K_tau,tau_of_K,    K_masses,Bm,zero_mat)
k4x = k4x - x; k4v = k4v - v

In [77]:
c5 = C[4]; a51 = A[4,0]; a52 = A[4,1]; a53 = A[4,2]; a54 = A[4,3]
k5x, k5v = f(t+c5*h, x+h*(a51*k1x + a52*k2x + a53*k3x + a54*k4x), v+h*(a51*k1v + a52*k2v + a53*k3v + a54*k4v),    K_tau,tau_of_K,    K_masses,Bm,zero_mat)
k5x = k5x - x; k5v = k5v - v

In [78]:
c6 = C[5]; a61 = A[5,0]; a62 = A[5,1]; a63 = A[5,2]; a64 = A[5,3]; a65 = A[5,4]
k6x, k6v = f(t+c6*h, x+h*(a61*k1x + a62*k2x + a63*k3x + a64*k4x + a65*k5x), v+h*(a61*k1v + a62*k2v + a63*k3v + a64*k4v + a65*k5v),    K_tau,tau_of_K,    K_masses,Bm,zero_mat)
k6x = k6x - x; k6v = k6v - v

In [86]:
#compute the result
x_out = x + h*(B[0]*k1x + B[1]*k2x + B[2]*k3x + B[3]*k4x + B[4]*k5x + B[5]*k6x)
v_out = v + h*(B[0]*k1v + B[1]*k2v + B[2]*k3v + B[3]*k4v + B[4]*k5v + B[5]*k6v)



In [87]:
#estimate the error of the result
x_err = x + h*(E[0]*k1x + E[1]*k2x + E[2]*k3x + E[3]*k4x + E[4]*k5x + E[5]*k6x)
x_err = x_err - x_out

v_err = v + h*(E[0]*k1v + E[1]*k2v + E[2]*k3v + E[3]*k4v + E[4]*k5v + E[5]*k6v)
v_err = v_err - v_out


In [91]:
max_err = np.max(np.abs(x_err))
mav_err = np.max(np.abs(v_err))

# Collect ^this into a single function and njit it

In [3]:
# # the stepsize
# h = 0.001

# t = 0.01
# K_index = 7
# Bm = element_array_inverse_equilibrium_position[K_index]
# Ka = element_array_index[K_index]
# x = vertices[Ka].copy()
# v = velocities[Ka].copy()
# tau_of_K = element_array_time[K_index]
# K_tau = node_array_time[Ka].copy()
# K_masses = element_array_mass[Ka]
# # step_to_time(t, K_vertices, K_velocities, K_masses, K_tau, tau_of_K, Bm, zero_mat)

def get_one_step_explicit_dormand_prince_method(mu,lam,gamma):
    """Explicit Runge-Kutta method of order 5(4).

    This uses the Dormand-Prince pair of formulas [1]_. The error is controlled
    assuming accuracy of the fourth-order method accuracy, but steps are taken
    using the fifth-order accurate formula (local extrapolation is done).
    A quartic interpolation polynomial is used for the dense output [2]_.

    Can be applied in the complex domain.

    Parameters
    ----------
    mu: the first Lamé parameter. nonnegative float.
    lam: the second Lamé parameter. nonnegative float.
    gamma: the Rayleigh damping parameter. nonnegative float

    References
    ----------
    .. [1] J. R. Dormand, P. J. Prince, "A family of embedded Runge-Kutta
           formulae", Journal of Computational and Applied Mathematics, Vol. 6,
           No. 1, pp. 19-26, 1980.
    .. [2] L. W. Shampine, "Some Practical Runge-Kutta Formulas", Mathematics
           of Computation,, Vol. 46, No. 173, pp. 135-150, 1986.
    """
    zero_mat = np.zeros((4,3))
    #one step implicit midpoint rule
    # local_one_step_implicit_midpoint_rule = get_local_one_step_implicit_midpoint_rule(mu,lam,gamma,num_iter = 30)
    #one step explicit Newmark method (misnamed, I know)
    compute_one_step_forward_euler_method=get_compute_one_step_forward_euler_method(mu,lam,gamma)
    step_to_time = compute_one_step_forward_euler_method

    @njit
    def f(t,x,v,K_tau,tau_of_K,K_masses,Bm,zero_mat):
        x_out, v_out = compute_one_step_forward_euler_method(t,x,v,K_masses,K_tau,tau_of_K,Bm,zero_mat)
        return x_out, v_out

    #from print(inspect.getsource(scipy.integrate.RK45))
    C = np.array([0, 1/5, 3/10, 4/5, 8/9, 1])
    A = np.array([
        [0, 0, 0, 0, 0],
        [1/5, 0, 0, 0, 0],
        [3/40, 9/40, 0, 0, 0],
        [44/45, -56/15, 32/9, 0, 0],
        [19372/6561, -25360/2187, 64448/6561, -212/729, 0],
        [9017/3168, -355/33, 46732/5247, 49/176, -5103/18656]
    ])
    B = np.array([35/384, 0, 500/1113, 125/192, -2187/6784, 11/84])
    E = np.array([-71/57600, 0, 71/16695, -71/1920, 17253/339200, -22/525,
                  1/40])
    @njit
    def foo(h,x,v,K_masses,K_tau,tau_of_K,Bm,zero_mat):
        """Explicit Runge-Kutta method of order 5(4).

            This uses the Dormand-Prince pair of formulas [1]_. The error is controlled
            assuming accuracy of the fourth-order method accuracy, but steps are taken
            using the fifth-order accurate formula (local extrapolation is done).
            A quartic interpolation polynomial is used for the dense output [2]_.

            Can be applied in the complex domain.

            Parameters
            ----------
            mu: the first Lamé parameter. nonnegative float.
            lam: the second Lamé parameter. nonnegative float.
            gamma: the Rayleigh damping parameter. nonnegative float

            References
            ----------
            .. [1] J. R. Dormand, P. J. Prince, "A family of embedded Runge-Kutta
                   formulae", Journal of Computational and Applied Mathematics, Vol. 6,
                   No. 1, pp. 19-26, 1980.
            .. [2] L. W. Shampine, "Some Practical Runge-Kutta Formulas", Mathematics
                   of Computation,, Vol. 46, No. 173, pp. 135-150, 1986.
            """
        #set the current time to the max of the configuration of the element
        t = max(np.max(K_tau), tau_of_K)

        #explicit newmark method to the current time
        k1x, k1v = f(t,x,v,    K_tau,tau_of_K,    K_masses,Bm,zero_mat)
        k1x = k1x - x; k1v = k1v - v

        c2 = C[1]; a21 = A[1,0]
        k2x, k2v = f(t+c2*h, x+h*a21*k1x, v+h*a21*k1v,    K_tau,tau_of_K,    K_masses,Bm,zero_mat)
        k2x = k2x - x; k2v = k2v - v

        c3 = C[2]; a31 = A[2,0]; a32 = A[2,1]
        k3x, k3v = f(t+c3*h, x+h*(a31*k1x + a32*k2x), v+h*(a31*k1v + a32*k2v),    K_tau,tau_of_K,    K_masses,Bm,zero_mat)
        k3x = k3x - x; k3v = k3v - v

        c4 = C[3]; a41 = A[3,0]; a42 = A[3,1]; a43 = A[3,2]
        k4x, k4v = f(t+c4*h, x+h*(a41*k1x + a42*k2x + a43*k3x), v+h*(a41*k1v + a42*k2v + a43*k3v),    K_tau,tau_of_K,    K_masses,Bm,zero_mat)
        k4x = k4x - x; k4v = k4v - v

        c5 = C[4]; a51 = A[4,0]; a52 = A[4,1]; a53 = A[4,2]; a54 = A[4,3]
        k5x, k5v = f(t+c5*h, x+h*(a51*k1x + a52*k2x + a53*k3x + a54*k4x), v+h*(a51*k1v + a52*k2v + a53*k3v + a54*k4v),    K_tau,tau_of_K,    K_masses,Bm,zero_mat)
        k5x = k5x - x; k5v = k5v - v

        c6 = C[5]; a61 = A[5,0]; a62 = A[5,1]; a63 = A[5,2]; a64 = A[5,3]; a65 = A[5,4]
        k6x, k6v = f(t+c6*h, x+h*(a61*k1x + a62*k2x + a63*k3x + a64*k4x + a65*k5x), v+h*(a61*k1v + a62*k2v + a63*k3v + a64*k4v + a65*k5v),    K_tau,tau_of_K,    K_masses,Bm,zero_mat)
        k6x = k6x - x; k6v = k6v - v

        #compute the result
        x_out = x + h*(B[0]*k1x + B[1]*k2x + B[2]*k3x + B[3]*k4x + B[4]*k5x + B[5]*k6x)
        v_out = v + h*(B[0]*k1v + B[1]*k2v + B[2]*k3v + B[3]*k4v + B[4]*k5v + B[5]*k6v)

        #estimate the error of the result
        x_err = x + h*(E[0]*k1x + E[1]*k2x + E[2]*k3x + E[3]*k4x + E[4]*k5x + E[5]*k6x)
        x_err = x_err - x_out
        v_err = v + h*(E[0]*k1v + E[1]*k2v + E[2]*k3v + E[3]*k4v + E[4]*k5v + E[5]*k6v)
        v_err = v_err - v_out

        max_err = np.max(np.abs(x_err))
        mav_err = np.max(np.abs(v_err))#*h

        return max_err, mav_err, x_out,v_out
    return foo

In [4]:
# foo(h,x,v,K_masses,K_tau,tau_of_K,Bm,zero_mat)

In [100]:
# foo(h,x,v,K_masses,K_tau,tau_of_K,Bm,zero_mat)

(3.4049460073681814e-07,
 1.536270950175389e-06,
 array([[ 0.62847645,  0.71433732, -0.26459579],
        [ 0.58759723,  0.5943788 , -0.47082971],
        [ 0.50580263,  0.413197  , -0.3059985 ],
        [ 0.38410106,  0.54358837, -0.43047151]]),
 array([[-0.64833983,  0.01337363,  0.01447463],
        [-0.57418608, -0.00598796, -0.03165533],
        [-0.50452982, -0.03965316,  0.0203962 ],
        [-0.46126644,  0.02161065, -0.00240728]]))

In [103]:
one_step_explicit_dormand_prince_method = get_one_step_explicit_dormand_prince_method(mu,lam,gamma)

In [104]:
one_step_explicit_dormand_prince_method(h,x,v,K_masses,K_tau,tau_of_K,Bm,zero_mat)

(3.4049460073681814e-07,
 1.536270950175389e-06,
 array([[ 0.62847645,  0.71433732, -0.26459579],
        [ 0.58759723,  0.5943788 , -0.47082971],
        [ 0.50580263,  0.413197  , -0.3059985 ],
        [ 0.38410106,  0.54358837, -0.43047151]]),
 array([[-0.64833983,  0.01337363,  0.01447463],
        [-0.57418608, -0.00598796, -0.03165533],
        [-0.50452982, -0.03965316,  0.0203962 ],
        [-0.46126644,  0.02161065, -0.00240728]]))

In [105]:
one_step_explicit_dormand_prince_method(h,x,v,K_masses,K_tau,tau_of_K,Bm,zero_mat)

(3.4049460073681814e-07,
 1.536270950175389e-06,
 array([[ 0.62847645,  0.71433732, -0.26459579],
        [ 0.58759723,  0.5943788 , -0.47082971],
        [ 0.50580263,  0.413197  , -0.3059985 ],
        [ 0.38410106,  0.54358837, -0.43047151]]),
 array([[-0.64833983,  0.01337363,  0.01447463],
        [-0.57418608, -0.00598796, -0.03165533],
        [-0.50452982, -0.03965316,  0.0203962 ],
        [-0.46126644,  0.02161065, -0.00240728]]))

# compute flow map and njit it

In [130]:
#TODO: update all nodes to max current time using current velocity
for a in range(4):
    x[a] += v[a] * (tau_of_K - K_tau[a]) #+ Delta_x[a]
        
#TODO: swap out f with compute force

In [136]:
Ds = get_D_mat(x)
W = get_element_volume(Ds)

In [137]:
force = compute_force(v, Ds, W, Bm, zero_mat.copy())

In [138]:
#def get_flow_map


In [139]:
dxdt =  v.copy()
dvdt =  v.copy()
for a in range(4):
    dvdt = force[a]/K_masses[a]


In [155]:
from lib.model.flow_map import *

In [156]:
compute_flow_map = get_flow_map(mu,lam,gamma)

In [157]:
dxdt, dvdt = compute_flow_map(x, v, Bm, K_masses)

In [159]:
dxdt, dvdt = compute_flow_map(x, v, Bm, K_masses)

In [4]:
minstepsize = 1e-8
h = 0.001
K_index = 7
one_step_explicit_dormand_prince_method = get_one_step_explicit_dormand_prince_method(mu,lam,gamma)


#get the element configuration
Bm = element_array_inverse_equilibrium_position[K_index]
Ka = element_array_index[K_index]
x = vertices[Ka].copy()
v = velocities[Ka].copy()
tau_of_K = element_array_time[K_index]
K_tau = node_array_time[Ka].copy()
K_masses = element_array_mass[Ka]
                

max_err, mav_err, x_out,v_out = one_step_explicit_dormand_prince_method(h,x,v,K_masses,K_tau,tau_of_K,Bm)

In [5]:
max_err, mav_err, x_out,v_out = one_step_explicit_dormand_prince_method(h,x,v,K_masses,K_tau,tau_of_K,Bm)

In [6]:
max_err, mav_err, x_out,v_out

(0.0006641166354861561,
 6.605173595158886e-05,
 array([[ 0.64727054,  0.71423555, -0.26470589],
        [ 0.60492409,  0.59442562, -0.47058825],
        [ 0.52079481,  0.41349968, -0.30615355],
        [ 0.39608137,  0.54342104, -0.43045433]]),
 array([[-6.47919052e-01,  1.35160740e-05,  1.46202271e-05],
        [-6.05499083e-01, -6.27847487e-06, -3.20940741e-05],
        [-5.21299591e-01, -4.02421571e-05,  2.05833027e-05],
        [-3.96540772e-01,  2.23621714e-05, -2.22679623e-06]]))

# test the method on the whole mesh

In [11]:
from lib.controller.integrate_forward_dormand_prince_synchronous import *

In [12]:
integrate_system_dormand_prince_synchronous = get_integrate_system_dormand_prince_synchronous(mu,lam,gamma)

In [13]:
tf = 0.00
retval = integrate_system_dormand_prince_synchronous(tf, element_array_time, element_array_stepsize, node_array_time,
                                         element_array_index, vertices, velocities, 
                                      node_array_mass, element_array_inverse_equilibrium_position, element_array_mass)#, minstepsize)

/Users/timothytyree/Documents/GitHub/avi/nb/lib/controller/integrate_forward_dormand_prince_synchronous.py:42: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (array(float64, 2d, C), array(float64, 2d, A))
  max_err, mav_err, x_out,v_out = one_step_explicit_dormand_prince_method(h,x,v,K_masses,K_tau,tau_of_K,Bm)


In [14]:
tf = 0.01
retval = integrate_system_dormand_prince_synchronous(tf, element_array_time, element_array_stepsize, node_array_time,
                                         element_array_index, vertices, velocities, 
                                      node_array_mass, element_array_inverse_equilibrium_position, element_array_mass)

In [18]:
tf = 0.02
retval = integrate_system_dormand_prince_synchronous(tf, element_array_time, element_array_stepsize, node_array_time,
                                         element_array_index, vertices, velocities, 
                                      node_array_mass, element_array_inverse_equilibrium_position, element_array_mass)

In [30]:
tf = 0.05
retval = integrate_system_dormand_prince_synchronous(tf, element_array_time, element_array_stepsize, node_array_time,
                                         element_array_index, vertices, velocities, 
                                      node_array_mass, element_array_inverse_equilibrium_position, element_array_mass)

### verify updates have happened in all relevant system degrees of freedom
retval

In [28]:
#before some computation
vertices[:5]

array([[ 0.3715328 ,  0.59476459,  0.68627453],
       [-0.93274623, -0.25802922, -0.16666667],
       [ 0.60552961,  0.59442562, -0.47058824],
       [-0.93884414,  0.17852867, -0.19607843],
       [-0.57890046, -0.02019236, -0.78431374]])

In [29]:
velocities[:5]

array([[-0.3715328 ,  0.        ,  0.        ],
       [ 0.93274623, -0.        , -0.        ],
       [-0.60552961,  0.        , -0.        ],
       [ 0.93884414,  0.        , -0.        ],
       [ 0.57890046, -0.        , -0.        ]])

In [25]:
#after some computation
vertices[:5]


array([[ 0.35597766,  0.59660302,  0.68880254],
       [-0.89175045, -0.25941481, -0.16659651],
       [ 0.57951287,  0.59724999, -0.47293893],
       [-0.89741359,  0.1792675 , -0.19699422],
       [-0.55534901, -0.01994079, -0.7902785 ]])

In [26]:
velocities[:5]

array([[-0.2082372 ,  0.09229777,  0.13877773],
       [ 0.65618679, -0.08156239,  0.01402417],
       [-0.40733818,  0.15081991, -0.12813179],
       [ 0.71214749,  0.03528751, -0.04993599],
       [ 0.31854025,  0.00777699, -0.32364673]])

In [31]:
element_array_time

array([0.049, 0.049, 0.049, ..., 0.049, 0.049, 0.049])

In [34]:
node_array_time[:5]

array([0.049, 0.049, 0.049, 0.049, 0.049])

In [35]:
vertices

array([[ 0.35597766,  0.59660302,  0.68880254],
       [-0.89175045, -0.25941481, -0.16659651],
       [ 0.57951287,  0.59724999, -0.47293893],
       ...,
       [ 0.31015978,  0.5374809 ,  0.21935358],
       [ 0.33646057,  0.64583056,  0.35449834],
       [ 0.37695007,  0.72395069,  0.23846033]])

In [37]:
tau
tf

0.05

In [ ]:
# #update a copy of all positions using current velocity to the observation time of the element
# x = vertices.copy()
# for a in range(N_vertices):
#     x[a] += velocities[a] * (tf - tau[a])